In [1]:
import pandas as pd
import numpy as np

In [2]:
df_digest_protein = pd.read_csv('data/uniprot/df_digest_protein.csv')

In [3]:
df_digest_protein

,PROTEIN,SEQUENCE,PROTEIN_FULL,SPECTRAL_CNT,SPECTRAL_CNT_MISS
0,sp|Q9NP61|ARFG3_HUMAN,MGDPSKQDILTIFKRLRSVPTNKVCFDCGAKNPSWASITYGVFLCI...,>sp|Q9NP61|ARFG3_HUMAN ADP-ribosylation factor...,M_2;G_206;D_0;P_5;S_0;K_0;Q_7;D_3;I_3;L_3;T_3;...,M_2;G_208;D_208;P_213;S_213;K_213;Q_217;D_217;...
1,sp|P36404|ARL2_HUMAN,MGLLTILKKMKQKERELRLLMLGLDNAGKTTILKKFNGEDIDTISP...,>sp|P36404|ARL2_HUMAN ADP-ribosylation factor-...,M_0;G_0;L_0;L_0;T_0;I_0;L_0;K_0;K_0;M_0;K_0;Q_...,M_0;G_0;L_0;L_0;T_0;I_0;L_0;K_0;K_2;M_2;K_2;Q_...
2,sp|Q99873|ANM1_HUMAN,MAAAEAANCIMENFVATLANGMSLQPPLEEVSCGQAESSEKPNAED...,>sp|Q99873|ANM1_HUMAN Protein arginine N-methy...,M_0;A_0;A_0;A_0;E_0;A_0;A_0;N_0;C_0;I_0;M_0;E_...,M_0;A_0;A_0;A_0;E_0;A_0;A_0;N_0;C_0;I_0;M_0;E_...
3,sp|P48047|ATPO_HUMAN,MAAPAVSGLSRQVRCFSTSVVRPFAKLVRPPVQVYGIEGRYATALY...,">sp|P48047|ATPO_HUMAN ATP synthase subunit O, ...",M_0;A_9;A_9;P_9;A_9;V_9;S_9;G_9;L_9;S_9;R_9;Q_...,M_0;A_0;A_0;P_0;A_0;V_0;S_0;G_0;L_0;S_0;R_0;Q_...
4,sp|P53634|CATC_HUMAN,MGAGPSLLLAALLLLLSGDGAVRCDTPANCTYLDLLGTWVFQVGSS...,>sp|P53634|CATC_HUMAN Dipeptidyl peptidase 1 O...,M_0;G_0;A_0;G_0;P_0;S_0;L_0;L_0;L_0;A_0;A_0;L_...,M_0;G_0;A_0;G_0;P_0;S_0;L_0;L_0;L_0;A_0;A_0;L_...
...,...,...,...,...,...
5848,sp|O00471|EXOC5_HUMAN,MATTAELFEEPFVADEYIERLVWRTPGGGSRGGPEAFDPKRLLEEF...,>sp|O00471|EXOC5_HUMAN Exocyst complex compone...,M_0;A_119;T_119;T_119;A_119;E_120;L_120;F_122;...,M_0;A_0;T_0;T_0;A_0;E_0;L_0;F_0;E_0;E_0;P_0;F_...
5849,sp|P39748|FEN1_HUMAN,MGIQGLAKLIADVAPSAIRENDIKSYFGRKVAIDASMSIYQFLIAV...,>sp|P39748|FEN1_HUMAN Flap endonuclease 1 OS=H...,M_10;G_19;I_9;Q_9;G_10;L_9;A_11;K_15;L_423;I_1...,M_1;G_11;I_11;Q_11;G_12;L_12;A_14;K_20;L_305;I...
5850,sp|Q9UKT5|FBX4_HUMAN,MAGSEPRSGTNSPPPPFSDWGRLEAAILSGWKTFWQSVSKERVART...,>sp|Q9UKT5|FBX4_HUMAN F-box only protein 4 OS=...,M_0;A_0;G_0;S_0;E_0;P_0;R_0;S_4;G_3;T_3;N_3;S_...,M_0;A_0;G_0;S_0;E_0;P_0;R_0;S_1;G_1;T_1;N_1;S_...
5851,sp|Q9Y680|FKBP7_HUMAN,MPKTMHFLFRFIVFFYLWGLFTAQRQKKEESTEEVKIEVLHRPENC...,>sp|Q9Y680|FKBP7_HUMAN Peptidyl-prolyl cis-tra...,M_0;P_0;K_0;T_0;M_0;H_0;F_0;L_0;F_0;R_0;F_0;I_...,M_0;P_0;K_0;T_0;M_0;H_0;F_0;L_0;F_0;R_0;F_0;I_...


# Digestibility

In [ ]:
def labeling(PC, PCmiss, thres = 4):
    data_dicKR = dict()
    for idx in range(len(PC)):
        for amino_idx in range(len(PC[idx])):
            if PC[idx][amino_idx][:1] in 'KR':
                # tryptic site extracting from protein
                # tryptic site in protein N-terminal
                if amino_idx <= thres - 1:
                    tryptic_site = 'Z'*(thres-amino_idx) + ''.join([a[0] for a in PC[idx][: amino_idx+thres + 1]])  # added Z
                # tryptic site in protein C-terminal
                elif amino_idx >= len(PC[idx]) - thres:
                    tryptic_site = ''.join([a[0] for a in PC[idx][amino_idx-thres: amino_idx+thres + 1]]) + 'Z'*(thres-(len(PC[idx])-1)+amino_idx)  # added Z
                else:
                    tryptic_site = ''.join([a[0] for a in PC[idx][amino_idx-thres: amino_idx+thres + 1]])

                # N-terminal of tryptic site condition
                N_PCondition = int(PC[idx][amino_idx][2:]) >= 1
                # C-terminal of tryptic site condition
                if amino_idx == len(PC[idx]) - 1:  # tryptic site in last position of protein
                    C_PCondition = True
                else:
                    C_PCondition = int(PC[idx][amino_idx + 1][2:]) >= 1
                # miss cleavage of tryptic site contidion
                miss_PCondition = int(PCmiss[idx][amino_idx][2:]) == 0
                # True of all above, Positive labeling
                P_condition = (N_PCondition == True) and (C_PCondition == True) and (miss_PCondition == True)

                # N-terminal of tryptic site condition
                N_NCondition = int(PC[idx][amino_idx][2:]) == 0
                # C-terminal of tryptic site condition
                if amino_idx == len(PC[idx]) - 1:  # tryptic site in last position of protein
                    C_NCondition = False
                else:
                    C_NCondition = int(PC[idx][amino_idx + 1][2:]) == 0
                # miss cleavage of tryptic site condition
                miss_Ncondition = int(PCmiss[idx][amino_idx][2:]) >= 1
                # True of all above, Negative labeling
                N_condition = (N_NCondition == True) and (C_NCondition == True) and (miss_Ncondition == True)

                # making data
                if P_condition:
                    if tryptic_site not in data_dicKR:
                        data_dicKR[tryptic_site] = []
                    data_dicKR[tryptic_site].append(('P', idx, amino_idx))
                elif N_condition:
                    if tryptic_site not in data_dicKR:
                        data_dicKR[tryptic_site] = []
                    data_dicKR[tryptic_site].append(('N', idx, amino_idx))
    p = [k for k, v in data_dicKR.items() if v[0][0]=='P']
    n = [k for k, v in data_dicKR.items() if v[0][0]=='N']
    print(' P, unique p: {} \n n, unique n: {} \n total : {:,}'.format((len(p), len(set(p))), (len(n), len(set(n))), len(p)+len(n)))
    return data_dicKR